# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [7]:
import pandas as pd

# Carregar o arquivo demo01.csv
df = pd.read_csv('demo01.csv')

# Verificar as variáveis (colunas)
print(df.columns)

# Verificar os tipos de dados de cada variável
print(df.dtypes)

# Verificar a quantidade de missings
missing_values = df.isnull().sum()
print(missing_values)

# Verificar a distribuição da variável resposta 'mau'
print(df['mau'].value_counts())


Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'possui_celular', 'possui_fone_comercial',
       'possui_fone', 'possui_email', 'qt_pessoas_residencia', 'mau'],
      dtype='object')
sexo                      object
posse_de_veiculo          object
posse_de_imovel           object
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                    float64
tempo_emprego            float64
possui_celular             int64
possui_fone_comercial      int64
possui_fone                int64
possui_email               int64
qt_pessoas_residencia    float64
mau                         bool
dtype: object
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao          

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [8]:
# Criar uma lista para armazenar os metadados
metadata = []

# Loop através das colunas para contar o número de categorias e os tipos
for col in df.columns:
    data_type = df[col].dtype
    if data_type == 'object':  # Para variáveis qualitativas
        num_categories = df[col].nunique()
    else:
        num_categories = None

    metadata.append({'Variable': col, 'Type': data_type, 'Num_Categories': num_categories})

# Criar o dataframe de metadados
metadata_df = pd.DataFrame(metadata)
print(metadata_df)


                 Variable     Type  Num_Categories
0                    sexo   object             2.0
1        posse_de_veiculo   object             2.0
2         posse_de_imovel   object             2.0
3              qtd_filhos    int64             NaN
4              tipo_renda   object             5.0
5                educacao   object             5.0
6            estado_civil   object             5.0
7         tipo_residencia   object             6.0
8                   idade  float64             NaN
9           tempo_emprego  float64             NaN
10         possui_celular    int64             NaN
11  possui_fone_comercial    int64             NaN
12            possui_fone    int64             NaN
13           possui_email    int64             NaN
14  qt_pessoas_residencia  float64             NaN
15                    mau     bool             NaN


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [9]:
# Criação das variáveis dummy
df_dummies = pd.get_dummies(df, drop_first=True)

# Verificar as primeiras linhas do novo dataframe
print(df_dummies.head())


   qtd_filhos      idade  tempo_emprego  possui_celular  \
0           0  58.832877       3.106849               1   
1           0  52.356164       8.358904               1   
2           0  52.356164       8.358904               1   
3           0  46.224658       2.106849               1   
4           0  29.230137       3.021918               1   

   possui_fone_comercial  possui_fone  possui_email  qt_pessoas_residencia  \
0                      0            0             0                    2.0   
1                      0            1             1                    1.0   
2                      0            1             1                    1.0   
3                      1            1             1                    2.0   
4                      0            0             0                    2.0   

     mau  sexo_M  ...  educacao_Secondary / secondary special  \
0  False    True  ...                                    True   
1  False   False  ...                         

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [10]:
# Transformar 'posse_de_veiculo' e 'possui_email' em variáveis numéricas (0 e 1)
df['posse_de_veiculo'] = df['posse_de_veiculo'].map({'N': 0, 'Y': 1})
df['possui_email'] = df['possui_email'].map({0: 0, 1: 1})

# Verificar a tabela cruzada
contingency_table = pd.crosstab(df['possui_email'], df['posse_de_veiculo'], margins=True)
print(contingency_table)

# Agora, calcular a correlação entre as variáveis e 'mau'
correlation_email = df[['possui_email', 'mau']].corr()
correlation_veiculo = df[['posse_de_veiculo', 'mau']].corr()

print("Correlação com possui_email:", correlation_email)
print("Correlação com posse_de_veiculo:", correlation_veiculo)


posse_de_veiculo      0     1    All
possui_email                        
0                  9341  5829  15170
1                   837   643   1480
All               10178  6472  16650
Correlação com possui_email:               possui_email       mau
possui_email      1.000000 -0.006512
mau              -0.006512  1.000000
Correlação com posse_de_veiculo:                   posse_de_veiculo       mau
posse_de_veiculo          1.000000 -0.007003
mau                      -0.007003  1.000000


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [11]:
# Salvar o dataframe modificado
df_dummies.to_csv('demo01_processed.csv', index=False)
